In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, measure, img_as_ubyte
from skimage.color import rgb2gray
from skimage.transform import resize
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram
import os

# Fonction pour extraire les caractéristiques d'une tache
def extract_features(region):
    # Calcul des propriétés de la région
    props = measure.regionprops(region.filled_image)[0]
    
    # Extraction des caractéristiques
    area = props.area
    perimeter = props.perimeter
    eccentricity = props.eccentricity
    extent = props.extent
    aspect_ratio = props.major_axis_length / props.minor_axis_length if props.minor_axis_length > 0 else 0
    
    return [aspect_ratio, eccentricity, extent, perimeter / np.sqrt(area)]

# Fonction pour traiter une image et extraire les caractéristiques de toutes les taches
def process_image(image_path):
    # Charger l'image
    img = io.imread(image_path)
    
    # Convertir en niveaux de gris si nécessaire
    if len(img.shape) > 2:
        img = rgb2gray(img)
    
    # Binariser l'image
    binary = img > 0
    
    # Étiqueter les régions connectées
    labeled = measure.label(binary)
    
    # Extraire les propriétés des régions
    regions = measure.regionprops(labeled)
    
    # Extraire les caractéristiques de chaque tache
    features = [extract_features(region) for region in regions]
    
    return np.array(features)

# Fonction principale pour analyser toutes les images et effectuer le clustering
def analyze_salamander_spots(folder_path):
    all_features = []
    image_names = []
    
    # Parcourir toutes les images dans le dossier
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            features = process_image(image_path)
            all_features.extend(features)
            image_names.extend([filename] * len(features))
    
    # Convertir en array numpy
    all_features = np.array(all_features)
    
    # Normaliser les caractéristiques
    all_features = (all_features - np.mean(all_features, axis=0)) / np.std(all_features, axis=0)
    
    # Calculer la matrice de distance
    distances = pdist(all_features)
    
    # Effectuer le clustering hiérarchique
    linkage_matrix = linkage(distances, method='ward')
    
    # Tracer le dendrogramme
    plt.figure(figsize=(10, 7))
    dendrogram(linkage_matrix)
    plt.title('Dendrogramme du clustering hiérarchique des taches de salamandres')
    plt.xlabel('Échantillon')
    plt.ylabel('Distance')
    plt.show()
    
    return all_features, image_names, linkage_matrix

# Appel de la fonction principale
folder_path = '../testing/Building_images'
features, image_names, linkage_matrix = analyze_salamander_spots(folder_path)

print(f"Nombre total de taches analysées : {len(features)}")
print(f"Nombre de caractéristiques par tache : {features.shape[1]}")